# Automation StoryMap Editing

### Import Libraries

In [ ]:
from arcgis.gis import GIS
from arcgis.apps.storymap import Image, StoryMap, Text, TextStyles
from arcgis.apps.storymap.story_content import Scales

In [ ]:
import json
import copy
import numpy as np
import pandas as pd

In [ ]:
gis = GIS(profile="your_online_portal")
gis

In [ ]:
file_path = r"C:\ipython_workfolder\Content"

## Get the template StoryMap

In [ ]:
california = StoryMap("3864da69d9654c7e9d87b9b47d18e891")
california

### Understanding Nodes

In [ ]:
california.nodes

##### A storymap is composed of nodes. These nodes represent different content within your story.  
##### We've encapsulated this within our methods so you can edit the content and created automated changes.

## Properties of nodes

In [ ]:
# Take a look at a simpler class
content = california.get("n-znyJMA")
print("Properties of " + str(content) + ": ")
print(content.properties)

In [ ]:
# What about a more complicated class
content = california.get("n-VLxymn")
print("Properties of " + str(content) + ": ")
print(content.properties)

In [ ]:
# Look at the structure in a cleaner way
for item in content.properties:
    print(item)

##### When you use a method to make changes, you are making changes to the underlying dictionary structure of the storymap.

## Reading and preparing our data

In [ ]:
state = "Washington"

In [ ]:
# created a csv with all my data
data = pd.read_csv(file_path + r"\Washington\WashingtonData.csv")
data

In [ ]:
# create class from content and add to the DataFrame
classes = []

for index, row in data.iterrows():
    
    trow = row["Type"]
    crow = row["Content"]
    
    if trow == "paragraph":
        content = Text(text = crow, style = TextStyles.PARAGRAPH)
    elif trow == "subheading":
        content = Text(text = crow, style = TextStyles.SUBHEADING)
    elif trow == "image":
        content = Image(crow)
    else:
        content = None
    classes.append(content)

data["Class"] = classes

In [ ]:
data

In [ ]:
# Each value of the class column represents a story content instance
data.iloc[0]["Class"]

## Duplicating the template story

In [ ]:
new_item = california.duplicate(title=state)
new_item

In [ ]:
type(new_item)

In [ ]:
# convert to an instance of storymap
story = StoryMap(new_item)
story

In [ ]:
# node ids have stayed the same
story.nodes

## Begin Editing
### First, the story cover

In [ ]:
cover_text = data.iloc[0]["Content"] #access the text in the first row of our dataframe
cover_image = Image(data.iloc[1]["Content"]) #create an instance of Image to be passed in

In [ ]:
# update the story cover
story.cover(title=state, summary = cover_text, image= cover_image)

### Editing the WebMap extent

In [ ]:
wmap = story.get("n-5TCKSm")
wmap.map

In [ ]:
wmap

In [ ]:
# get the extent from our data
for index, row in data.iterrows():
    if row["Type"] == "extent":
        extent = json.loads(row["Content"]) #turn string to dict

extent

In [ ]:
# Use set_viewpoint method on the map instance
wmap.set_viewpoint(extent)

### Save to see our progress so far

In [ ]:
story.save()

### Update the timeline with our current data

In [ ]:
timeline = story.get("n-qiYJn0")
timeline

In [ ]:
# get the data for the timeline from the DataFrame
timeline_data = data[data["For"] == "timeline"]
timeline_data

In [ ]:
timeline.properties

In [ ]:
# list of the events that will be edited
events = np.repeat([1,2,3,4,5,6,7,8],2)
idx = 0
for event in events:
    timeline.edit(content=timeline_data.iloc[idx]["Class"], event=event)
    idx = idx + 1

timeline.properties

### Update the gallery with our current data

In [ ]:
gallery_data = data[data["For"] == "gallery"]
gallery_data

In [ ]:
new_images = gallery_data["Class"].tolist()
new_images

In [ ]:
gallery = story.get("n-ddtT6K")
gallery.properties

In [ ]:
# First, delete the current images so we only have the new ones once added
current = copy.copy(gallery.images)

for node in current:
    gallery.delete_image(node)
gallery.images

In [ ]:
gallery.add_images(new_images)
gallery.images

### Update the sidecar with our current data

In [ ]:
sidecar_data = data[data["For"] == "sidecar"]
sidecar_data

In [ ]:
extent

In [ ]:
texts = sidecar_data["Content"].tolist()
texts.pop(0) #remove extent
for text in texts:
    print(text)

In [ ]:
sidecar = story.get("n-VLxymn")
sidecar.properties

In [ ]:
sidecar.get("n-nC5ToS")

In [ ]:
idx = 0
for slide_dict in sidecar.properties[1:]:
    for slide in slide_dict:
        # Getting the map content
        wm_id = slide_dict[slide]["media"]["webmap"]
        webmap = sidecar.get(wm_id)
        # Change the extent
        webmap.set_viewpoint(extent, Scales.STATE)

        # Getting the text content
        txt_id = slide_dict[slide]["narrative_panel"]["children"]["text"]
        text = sidecar.get(txt_id)
        # Change the text
        text.text = texts[idx]
        idx += 1

sidecar.properties

In [ ]:
story.save()